In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import math

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform

import plotly.express as px
from sklearn.decomposition import PCA

In [ ]:
# Setup
sns.set_style("whitegrid")

abs_l2fc_threshold = np.log2(1.5)
padj_threshold = 0.05
outdir = 'volcano_plot_results'
image_formats = ('png', 'svg', 'eps')

gene_lookup_file = '../rsap_results/expression_data_pipeline_format/gene_id_gene_name_lookup_table.tsv.gz'

deseq_data_file = '../rsap_results/deseq2/comparison2__COMPETENCE2_Sex__Not_competent_vs_Competent/Comparison_Competent_vs_Not_competent/Comparison_Competent_vs_Not_competent.deseq2_results.tsv'    
#deseq_data_file = '../rsap_results/deseq2/comparison3__TREATED_Cell_line__Not_treated_vs_Treated/Comparison_Treated_vs_Not_treated/Comparison_Treated_vs_Not_treated.deseq2_results.tsv'

In [ ]:
print(f'Reading in DESeq2 data file: {deseq_data_file}')
deseq_data= pd.read_csv(deseq_data_file, sep='\t')
print(f'{deseq_data.shape[0]} genes reported')

if gene_lookup_file is not None:
    print(f'Reading in gene lookups file: {gene_lookup_file}')
    gene_lookups = pd.read_csv(gene_lookup_file, sep='\t')
    print(f'{gene_lookups.shape[0]} gene lookups reported')


In [ ]:
#Format
deseq_data['minus_log10(padj)'] = -np.log10(deseq_data['padj'])

In [ ]:
deseq_data.head(2)

In [ ]:
volcano_data = deseq_data.loc[:, ['region', 'log2FoldChange', 'minus_log10(padj)']]

# Remove entries containing NA
print(f'{volcano_data.shape[0]} genes BEFORE filter for NA')
filt = ~ volcano_data.isna().any(axis=1)
volcano_data = volcano_data[filt]
print(f'{volcano_data.shape[0]} genes AFTER filtering for NA')

In [ ]:
volcano_data.head(2)

In [ ]:
volcano_data['Significant'] = 'NO'

filt = (volcano_data['log2FoldChange'] >= abs_l2fc_threshold) & (volcano_data['minus_log10(padj)'] >= -np.log10(padj_threshold))
volcano_data.loc[filt, 'Significant'] = 'UP'

filt = (volcano_data['log2FoldChange'] <= -abs_l2fc_threshold) & (volcano_data['minus_log10(padj)'] >= -np.log10(padj_threshold))
volcano_data.loc[filt, 'Significant'] = 'DOWN'

In [ ]:
log2FoldChange_off_scale = 10
minus_log10_padj_off_scale = 10

volcano_data['Off_scale'] = False

filt = volcano_data['log2FoldChange'] > log2FoldChange_off_scale
volcano_data.loc[filt, 'log2FoldChange'] = log2FoldChange_off_scale
volcano_data.loc[filt, 'Off_scale'] = True

filt = volcano_data['log2FoldChange'] < -log2FoldChange_off_scale
volcano_data.loc[filt, 'log2FoldChange'] = -log2FoldChange_off_scale
volcano_data.loc[filt, 'Off_scale'] = True

filt = volcano_data['minus_log10(padj)'] > minus_log10_padj_off_scale
volcano_data.loc[filt, 'minus_log10(padj)'] = minus_log10_padj_off_scale
volcano_data.loc[filt, 'Off_scale'] = True

filt = volcano_data['minus_log10(padj)'] < -minus_log10_padj_off_scale
volcano_data.loc[filt, 'minus_log10(padj)'] = -minus_log10_padj_off_scale
volcano_data.loc[filt, 'Off_scale'] = True


volcano_data = volcano_data.reset_index(drop=True) #The needs doing

In [ ]:
# Make output directory
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [ ]:
# Make volcano plot with no annotations
if volcano_data['Off_scale'].sum() > 0:   # Prevents error
    markers = ['o', '*']
else:
    markers = ['o']


colors = ["black", "red", 'blue']
sns.set_palette(sns.color_palette(colors))

sns.scatterplot(data=volcano_data, 
                x="log2FoldChange", 
                y="minus_log10(padj)", 
                hue="Significant",
                hue_order=['NO', 'UP', 'DOWN'],
                style="Off_scale",
                markers=markers,
                s=7,
                edgecolor = None
               )
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
#plt.title(comparison)
#plt.axhline(y = -np.log10(padj_threshold), color = 'r', linestyle = '--', lw=0.5) 
#plt.axvline(x = abs_l2fc_threshold, color = 'r', linestyle = '--', lw=0.5) 
#plt.axvline(x = -abs_l2fc_threshold, color = 'r', linestyle = '--', lw=0.5) 

outfile = f'{outdir}/{os.path.basename(deseq_data_file)}.padj{padj_threshold}_abs_l2fc_{abs_l2fc_threshold}_volcano_plot'
for image_format in image_formats:
    plt.savefig(fname=f'{outfile}.{image_format}', bbox_inches='tight', pad_inches=0.5)
#plt.clf()

In [ ]:
# Print the name of the genes with most significant changes
volcano_data['volcano_weighting'] = (abs(volcano_data['log2FoldChange'])**2) * volcano_data['minus_log10(padj)']

filt = volcano_data['Significant'] == 'NO'   # Only allow singificant genes to be names
volcano_data.loc[filt, 'volcano_weighting'] = 0

volcano_data['volcano_weighting'] = volcano_data['volcano_weighting'].rank(ascending=False)

In [ ]:
# Use the look-up file, if provided
if gene_lookup_file is not None:
    gene_lookups = gene_lookups.rename(mapper={'gene_id' : 'region', 'gene_name' : 'gene'}, axis=1)
    volcano_data = pd.merge(volcano_data, gene_lookups, on='region', how='left')
else:
    volcano_data = volcano_data.rename(mapper={'region' : 'gene'})


In [ ]:
number_annotations = 35

sns.set_style("whitegrid")

sns.scatterplot(data=volcano_data, 
                x="log2FoldChange", 
                y="minus_log10(padj)", 
                hue="Significant",
                hue_order=['NO', 'UP', 'DOWN'],
                style="Off_scale",
                markers=markers,
                s=7,
                edgecolor = None
               )
for i in range(volcano_data.shape[0]):
    if volcano_data.loc[i, 'volcano_weighting'] <= number_annotations:
        plt.text(x=volcano_data.loc[i, 'log2FoldChange'] + 0.1,
                 y=volcano_data.loc[i, 'minus_log10(padj)'] + 0.1,
                 s=volcano_data.loc[i, 'gene'], 
                 fontsize=6
                 #fontdict=dict(color='red',size=10),
                 #bbox=dict(facecolor='yellow',alpha=0.5)
                )


plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

outfile = f'{outdir}/{os.path.basename(deseq_data_file)}.padj{padj_threshold}_abs_l2fc_{abs_l2fc_threshold}_volcano_plot_annotated'
for image_format in image_formats:
    plt.savefig(fname=f'{outfile}.{image_format}', bbox_inches='tight', pad_inches=0.5)

In [ ]:
# Write DE genes to file
abs_l2fc_threshold = np.log2(1.5)
padj_threshold = 0.05


# Make output directory
outdir = 'de_genes'
if not os.path.exists(outdir):
    os.makedirs(outdir)

for change in ('UP', 'DOWN'):
    outfile = f'{outdir}/{os.path.basename(deseq_data_file)}.{change}_de_genes_padj{padj_threshold}_abs_l2fc_{abs_l2fc_threshold}.txt'
    changing_genes = volcano_data.query('Significant == @change')['region']
    print(f'{len(changing_genes)} {change} DEGs')
    changing_genes.to_csv(outfile, sep='\t', index=False, header=False)

In [ ]:
print('Done')